In [16]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
 , preprocess_string, strip_short, stem_text
from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

In [3]:
path_to_csv = r"../input/Musical_instruments_reviews.csv"
# load data
df = pd.read_csv(path_to_csv, usecols=['reviewerID', 'reviewText'])
df.head()

,reviewerID,reviewText
0,A2IBPI20UZIR0U,"Not much to write about here, but it does exac..."
1,A14VAT5EAX3D9S,The product does exactly as it should and is q...
2,A195EZSQDW3E21,The primary job of this device is to block the...
3,A2C00NNG1ZQQG2,Nice windscreen protects my MXL mic and preven...
4,A94QU4C90B1AX,This pop filter is great. It looks and perform...


In [5]:
df=df.dropna()
df.head()

,reviewerID,reviewText
0,A2IBPI20UZIR0U,"Not much to write about here, but it does exac..."
1,A14VAT5EAX3D9S,The product does exactly as it should and is q...
2,A195EZSQDW3E21,The primary job of this device is to block the...
3,A2C00NNG1ZQQG2,Nice windscreen protects my MXL mic and preven...
4,A94QU4C90B1AX,This pop filter is great. It looks and perform...


In [9]:
# preprocess given text
def preprocess(text):
    #clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(),
                    remove_stopwords,
                    strip_punctuation,
                    strip_short,
                    stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)

    return text

In [11]:
# apply function to all reviews
df['Text (Clean)'] = df['reviewText'].apply(lambda x: preprocess(x))
df.head()

,reviewerID,reviewText,Text (Clean)
0,A2IBPI20UZIR0U,"Not much to write about here, but it does exac...","[write, here, exactli, suppos, filter, pop, so..."
1,A14VAT5EAX3D9S,The product does exactly as it should and is q...,"[product, exactli, afford, realiz, doubl, scre..."
2,A195EZSQDW3E21,The primary job of this device is to block the...,"[primari, job, devic, block, breath, produc, p..."
3,A2C00NNG1ZQQG2,Nice windscreen protects my MXL mic and preven...,"[nice, windscreen, protect, mxl, mic, prevent,..."
4,A94QU4C90B1AX,This pop filter is great. It looks and perform...,"[pop, filter, great, look, perform, like, stud..."


In [15]:
# create a dictionary with the corpus
corpus = df['Text (Clean)']
dictionary = corpora.Dictionary(corpus)
# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

In [17]:
# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.45969944165248133
Coherence score with 3 clusters: 0.4232693521008318
Coherence score with 4 clusters: 0.4419457521288399
Coherence score with 5 clusters: 0.3976642819896913
Coherence score with 6 clusters: 0.4140032803269855
Coherence score with 7 clusters: 0.41753381878886847
Coherence score with 8 clusters: 0.40298612939075584
Coherence score with 9 clusters: 0.40294102103414214
Coherence score with 10 clusters: 0.3792786830984747


In [22]:
# perform SVD on the bag of words with the LsiModel to extract 2 topics
lsi = LsiModel(bow, num_topics=2, id2word=dictionary)
# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.329*"sound" + 0.314*"guitar" + 0.242*"string" + 0.232*"pedal" + 0.217*"amp".
Words in 1: -0.584*"string" + 0.428*"pedal" + -0.380*"guitar" + 0.312*"amp" + -0.161*"tune".


In [24]:
# find the scores given between the review and each topic
corpus_lsi = lsi[bow]
score1 = []
score2 = []
for doc in corpus_lsi:
    score1.append(round(doc[0][1],2))
    score2.append(round(doc[1][1],2))
# create data frame that shows scores assigned for both topics for each review
df_topic = pd.DataFrame()
df_topic['Text'] = df['reviewText']
df_topic['Topic 0 score'] = score1
df_topic['Topic 1 score'] = score2
df_topic['Topic']= df_topic[['Topic 0 score', 'Topic 1 score']].apply(lambda x: x.argmax(), axis=1)
df_topic.head(5)

,Text,Topic 0 score,Topic 1 score,Topic
0,"Not much to write about here, but it does exac...",0.88,0.22,0
1,The product does exactly as it should and is q...,1.19,0.08,0
2,The primary job of this device is to block the...,1.03,0.21,0
3,Nice windscreen protects my MXL mic and preven...,0.38,-0.02,0
4,This pop filter is great. It looks and perform...,0.84,0.20,0


In [25]:
# find a sample review from each topic
df_topic0 = df_topic[df_topic['Topic'] == 0]
df_topic1 = df_topic[df_topic['Topic']==1]
print('Sample text from topic 0:\n {}'.format(df_topic0.sample(1, random_state=2)['Text'].values))
print('\nSample text from topic 1:\n {}'.format(df_topic1.sample(1, random_state=2)['Text'].values))

Sample text from topic 0:
 ["If you need a tube screamer. You can't go wrong with the TS808. The only reason why you shouldn't get it, is if it isn't the right tone for you. I say this because you can't say anything bad about this pedal. It sounds amazing. It just might not be your sound. I personally went with the TS808 because it compliments my amp better than the TS9. People who say this is better than the TS9 or vice versa are choosing the wrong wording. They prefer it. It's not better. Both are great."]

Sample text from topic 1:
 ["While this is not a $300 tape delay...it's a solid pedal that gets it done. It's way better built than other pedals in this price range. I'm sold. I've purchased several JoYo pedals and all of them are built extremely well and perform as good as if not better than top name pedals."]
